In [61]:

import pandas
from keras.models import Sequential
from keras.layers import Dense

# mlp for multi-label classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler

In [62]:

dataframe = pandas.read_csv("persistence/training.csv")
dataset = dataframe.drop('awardedMatches', axis=1).drop('id', axis=1).values
X = dataset[:,0:98].astype(float)
y = dataset[:, 98:].astype(int)
print(X.shape, y.shape)

(201, 98) (201, 3)


In [63]:

# define the model
model = Sequential()
model.add(Dense(90, input_dim=98, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X, y, epochs=150, batch_size=10)

Epoch 1/150
21/21 [==============================] - 1s 2ms/step - loss: 100.9989
Epoch 2/150
21/21 [==============================] - 0s 2ms/step - loss: 46.1399
Epoch 3/150
21/21 [==============================] - 0s 2ms/step - loss: 34.0369
Epoch 4/150
21/21 [==============================] - 0s 2ms/step - loss: 28.2923
Epoch 5/150
21/21 [==============================] - 0s 3ms/step - loss: 28.7136
Epoch 6/150
21/21 [==============================] - 0s 2ms/step - loss: 20.1732
Epoch 7/150
21/21 [==============================] - 0s 2ms/step - loss: 19.9240

In [64]:

...
results = list()
n_inputs, n_outputs = X.shape[1], y.shape[1]
# define evaluation procedure
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# enumerate folds
for train_ix, test_ix in cv.split(X):
	# prepare data
	X_train, X_test = X[train_ix], X[test_ix]
	y_train, y_test = y[train_ix], y[test_ix]
	st_x= MinMaxScaler()
	X_train= st_x.fit_transform(X_train)
	X_test= st_x.transform(X_test)
	# fit model
	model.fit(X_train, y_train, verbose=0, epochs=100)
	# make a prediction on the test set
	yhat = model.predict(X_test)
	# print(yhat)
	# print(y_test)
	# round probabilities to class labels
	yhat = yhat.round()
	# calculate accuracy
	acc = accuracy_score(y_test, yhat)
	# store result
	print('>%.2f' % acc)
	results.append(acc)

>0.43
>0.35
>0.25
>0.30
>0.25
>0.25
>0.45
>0.40
>0.50
>0.30
>0.29
>0.55
>0.60
>0.40
>0.60
>0.70
>0.50
>0.65
>0.70
>0.75
>0.71
>0.90
>0.90
>0.95
>0.90
>0.95
>0.95
>1.00
>1.00
>1.00


In [65]:
print('Average accuracy: %.2f%% \nStandart deviation: %.2f%%' % ((mean(results))*100, (std(results)*100)))

Average accuracy: 61.60% 
Standart deviation: 26.02%
